In [7]:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.power import tt_ind_solve_power

# Кейс

Команда монетизации тестирует новую систему цен на очень небольшую аудиторию. Аудитория достаточно пассивная, поэтому в ней редко прокрашиваются фичи. Основная метрика ARPPU.
Необходимо провести первичный анализ и найти наблюдаемый эффект между группами, проанализировать базовый A/B-тест и понять есть ли стат. значимое изменение в post данных после внедрения новых цен, провести CUPED t-test, рассчитать размер выборки для CUPED t-test.

# Загрузка данных

Данные содержат поюзерную пару ARPPU до теста (pre_ARPPU) и ARPU после тесте (post_ARPU)

In [9]:
df_cuped = pd.read_csv('cuped.csv')
df_cuped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4460 entries, 0 to 4459
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   post_ARPPU  4460 non-null   float64
 1   pre_ARPPU   3867 non-null   float64
 2   group       4460 non-null   object 
dtypes: float64(2), object(1)
memory usage: 104.7+ KB


In [11]:
df_cuped.head()

,post_ARPPU,pre_ARPPU,group
0,660.0,595.0,A
1,540.0,621.0,A
2,863.0,782.0,A
3,431.0,567.0,A
4,434.0,473.0,A


# Первичный анализ

In [13]:
# Отфильтруем данные по группам A и B
group_a = df_cuped[df_cuped['group'] == 'A']['post_ARPPU']
group_b = df_cuped[df_cuped['group'] == 'B']['post_ARPPU']

# Рассчитаем средние значения
mean_a = group_a.mean()
mean_b = group_b.mean()

# Абсолютная разница между средними
abs_diff = abs(mean_b - mean_a)

print(f"Среднее по группе A: {mean_a:.4f}")
print(f"Среднее по группе B: {mean_b:.4f}")
print(f"Абсолютная разница между средними: {abs_diff:.4f}")

Среднее по группе A: 598.6899
Среднее по группе B: 604.0458
Абсолютная разница между средними: 5.3559


In [15]:
# Стандартное отклонение pre_ARPPU в группе A
std_pre_a = df_cuped[df_cuped['group'] == 'A']['pre_ARPPU'].std(ddof=1)

# Расчёт эффекта (Cohen's d)
cohen_d = abs_diff / std_pre_a

print(f"Стандартное отклонение pre_ARPPU в группе A: {std_pre_a:.4f}")
print(f"Наблюдаемый эффект (Cohen's d): {cohen_d:.4f}")

Стандартное отклонение pre_ARPPU в группе A: 98.8946
Наблюдаемый эффект (Cohen's d): 0.0542


# Простой t-test

In [71]:
# Выполняем t-тест 
t_stat, p_value = stats.ttest_ind(group_b, group_a, equal_var=False)

print(f"T-статистика: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")

T-статистика: 1.7853
P-value: 0.0743


# CUPED t-test

In [19]:
# Общее количество строк
total_rows = len(df_cuped)

# Количество пропущенных значений в pre_ARPPU
missing_pre = df_cuped['pre_ARPPU'].isna().sum()

# Доля пропущенных значений
missing_ratio = missing_pre / total_rows

print(f"Доля пропущенных значений в pre_ARPPU: {missing_ratio:.4}")

Доля пропущенных значений в pre_ARPPU: 0.133


In [21]:
# Вычисляем среднее значение по pre_ARPPU (без NaN)
mean_pre = df_cuped['pre_ARPPU'].mean()

# Заполняем пропущенные значения этим средним
df_cuped['pre_ARPPU'] = df_cuped['pre_ARPPU'].fillna(mean_pre)

In [23]:
# Разобьем на соответствующие группы для того чтобы провести cuped t-test:
pre_control = df_cuped['pre_ARPPU'][df_cuped['group'] == 'A']
post_control = df_cuped['post_ARPPU'][df_cuped['group'] == 'A']
pre_test = df_cuped['pre_ARPPU'][df_cuped['group'] == 'B']
post_test = df_cuped['post_ARPPU'][df_cuped['group'] == 'B']

In [25]:
# Напишем функции для расчета theta и проведения cuped t-test:
def calculate_theta_for_test(control_pre, control_post, test_pre, test_post):
    theta = (np.cov(control_post, control_pre)[0, 1] + np.cov(test_post, test_pre)[0, 1]) /\
            (np.var(control_pre) + np.var(test_pre))
    return(theta)

def get_basic_ttest(group_A, group_B):
    t_stat, p_value = stats.ttest_ind(group_A, group_B)
    inference = {'t_stat': t_stat, 'p_value':p_value}
    return(inference)

def get_cuped_ttest(control_pre, control_post, test_pre, test_post):
    '''Проверяет гипотезу о равенстве средних CUPED вариант
    return - t_stat, p_value'''

    theta = calculate_theta_for_test(control_pre, control_post, test_pre, test_post)

    control_cuped = control_post - theta * control_pre
    test_cuped = test_post - theta * test_pre

    inference = get_basic_ttest(control_cuped, test_cuped)

    return(inference)

In [27]:
# Проведем cuped ttest:
cuped_ttest_inference = get_cuped_ttest(pre_control, post_control, pre_test, post_test)
print('p-value в cuped t-test', round(cuped_ttest_inference['p_value'], 4))

p-value в cuped t-test 0.0317


# Размер выборки

In [30]:
df_cuped_pre = pd.read_csv('cuped_pre_pre_data.csv')
df_cuped_pre.head()

,post_ARPPU,pre_ARPPU
0,629.0,620.0
1,536.0,619.0
2,503.0,596.0
3,649.0,578.0
4,667.0,752.0


In [32]:
# Вычисляем среднее значение(без NaN)
mean_pre = df_cuped_pre['pre_ARPPU'].mean()
mean_post = df_cuped_pre['post_ARPPU'].mean()

# Заполняем пропущенные значения этим средним
df_cuped_pre['pre_ARPPU'] = df_cuped_pre['pre_ARPPU'].fillna(mean_pre)
df_cuped_pre['post_ARPPU'] = df_cuped_pre['post_ARPPU'].fillna(mean_post)

In [34]:
def calculate_theta_basic(pre, post):
    theta = (np.cov(post, pre)[0, 1]) /\
            (np.var(pre))
    return(theta)
    
theta = calculate_theta_basic(df_cuped_pre['pre_ARPPU'], df_cuped_pre['post_ARPPU'])
print('Theta =', theta)

Theta = 0.6096554199736017


In [36]:
#стандартное отклонение Y_cuped
Y_cuped = df_cuped_pre['post_ARPPU'] - theta * df_cuped_pre['pre_ARPPU']
Y_cuped_std = Y_cuped.std()
print('Cтандартное отклонение Y_cuped =', Y_cuped_std)

Cтандартное отклонение Y_cuped = 83.47180548084842


In [38]:
#стандартное отклонение post_ARPPU
post_ARPPU_std = df_cuped_pre['post_ARPPU'].std()
print('Cтандартное отклонение post_ArPPU =', post_ARPPU_std)

Cтандартное отклонение post_ArPPU = 101.24946625472516


In [44]:
# Расчёт процентного уменьшения
reduction_percent = ((post_ARPPU_std - Y_cuped_std) / post_ARPPU_std) * 100
print(f'Cтандартное отклонение Y_cuped меньше стандартного отклонения post_ARPPU на {reduction_percent:.3f}', '%')

Cтандартное отклонение Y_cuped меньше стандартного отклонения post_ARPPU на 17.558 %


In [52]:
# рассчитаем контрольные средние и std. Берем post период для обычной выборки
control_std = df_cuped_pre['post_ARPPU'].std() # в функции generate_corr_data генерируются нормальные величины с std = 1
control_mean = df_cuped_pre['post_ARPPU'].mean()
mean_diff = control_mean * rel_effect

# Расчет индекса Коэна
cohen_d_basic  = mean_diff / control_std
print(cohen_d_basic)
n = tt_ind_solve_power(effect_size = cohen_d_basic,
                       alpha = alpha,
                       power = power,
                       ratio = 1,
                       alternative = "two-sided")
basic_sample_size = round(n)
print('Basic sample size =', basic_sample_size)

cohen_d_cuped  = mean_diff / Y_cuped_std
print(cohen_d_cuped)
n_cuped = tt_ind_solve_power(effect_size = cohen_d_cuped,
                       alpha = alpha,
                       power = 0.8,
                       ratio = 1,
                       alternative = "two-sided")
cuped_sample_size = round(n_cuped)
print('CUPED sample size =', cuped_sample_size)


0.05924643319868699
Basic sample size = 4473
0.07186462188408765
CUPED sample size = 3040
